In [ ]:
# Author: Joshua Gregory
# Date: Sept. 2025
# Descr: Uses BFS & DFS (Iteratively Deepening) to solve 8-Puzzle Problem variant.

In [1]:
from collections import deque
import random

In [2]:
GOAL_STATE = [[1,2,3], [8,0,4], [7,6,5]]  # solution state as 2d arr (0 = blank)
N = 3                                     # size of state space = NxN

row_moves = [0, 0, -1, 1]    # L/R/U/D values for row movements
col_moves = [-1, 1, 0, 0]    # L/R/U/D values for col movements

In [3]:
# Puzzle state space, as 2d arr = [[0,1,2], [3,4,5], [6,7,8]]
class EightPuzzle:
  def __init__(self, grid_state, blank_x, blank_y, depth_level):
    """
    Constructor for EightPuzzle objects.
    @param grid_state: attribute for 2d array state space.
    @param blank_x: attribute for row coord of blank tile.
    @param blank_y: attribute for column coord of blank tile.
    @param depth_level: attribute for depth of current state node.
    """
    self.grid_state = grid_state      # state space 2d array
    self.blank_x = blank_x            # x (row) coord of 0
    self.blank_y = blank_y            # y (col) coord of 0
    self.depth_level = depth_level    # num moves from root

In [4]:
# Generate root state space
def get_root_state():
  """
  Creates 8-puzzle root state space as a 2d array with elements [0,8] shuffled.
  @return: Root state of 8-puzzle grid, starting x & y coords of blank tile (0).
  """
  tile_values = list(range(9))  # list of integers [0,8], 0 = blank tile
  random.shuffle(tile_values)   # shuffle tile positions

  root_grid = [tile_values[i : i+3] for i in range(0, 9, 3)]  # convert to grid

  for blank_x in range(3):
    for blank_y in range(3):                  # search each col in each row
      if root_grid[blank_x][blank_y] == 0:    # for blank tile with value 0
        return root_grid, blank_x, blank_y    # return grid & xy coords of blank tile

In [5]:
# Check if goal state reached
def state_equals_goal(curr_state):
  """
  Checks if current state grid elements match those in GOAL_STATE.
  @return: (Boolean) T if all elements in curr_state match GOAL_STATE, else F.
  """
  return curr_state == GOAL_STATE

In [6]:
# Validate move is legal
def is_legal_move(adj_x, adj_y):
  """
  Validates that a tile position is non-diagonally adjacent to blank tile.
  @return: (Boolean) T if tile in range & adjacent to blank tile, else F.
  """
  return 0 <= adj_x < N and 0 <= adj_y < N

In [7]:
# Print grid state to console
def to_grid(grid_state):
  """
  Prints current grid state as a grid with an underscore for blank tile.
  @param grid_state: Current grid state represented as a 2d array.
  """
  print("--------")
  for row in grid_state:
    print(' '.join('_' if val == 0 else str(val) for val in row))
  print("--------")

In [8]:
# Swap blank tile using row_moves/col_moves
def get_new_state(grid_state, blank_x, blank_y, adj_x, adj_y):
  """
  Updates state space via swapping blank tile & adjacent tile values.
  @param grid_state: Current state of 8-puzzle grid.
  @param blank_x, blank_y: Current blank tile xy grid coords.
  @param adj_x, adj_y: Adjacent tile coords swapping values with blank tile.
  @return: 8-puzzle state after swapping tiles.
  """
  new_state = [row[:] for row in grid_state]             # copy current state grid

  temp = new_state[blank_x][blank_y]                     # set to temp var
  new_state[blank_x][blank_y] = new_state[adj_x][adj_y]  # swap with adj tile
  new_state[adj_x][adj_y] = temp                         # swap with blank tile

  return new_state                                       # return updated grid

In [9]:
def bfs_solver(grid_state, blank_x, blank_y):
  """
  Searches complete tiers of state nodes before visiting children nodes.
  @param state: Current state of 8-puzzle grid.
  @param x, y: Row, Column for current position of blank tile.
  """
  Q = deque()   # FIFO queue for searching complete tiers
  SEEN = set()  # re-init per instance, prevents cycling

  Q.append(EightPuzzle(grid_state, blank_x, blank_y, 0))  # init root & enqueue
  SEEN.add(tuple(map(tuple, grid_state)))  # convert to tuple, add to visited nodes

  while Q:                  # while states remain in queue
    current = Q.popleft()   # pop firstmost node

    print(f"\nDepth = {current.depth_level}:")  # print current depth
    to_grid(current.grid_state)                 # print current state as NxN grid

    if state_equals_goal(current.grid_state):
      print(f"\n~ Goal state reached at depth {current.depth_level}! ~")  # solution state reached
      return  # exit

    for i in range(4):  # move blank tile
      adj_x = current.blank_x + row_moves[i]  # new x = current + L/R/U/D
      adj_y = current.blank_y + col_moves[i]  # new y = current + L/R/U/D

      if is_legal_move(adj_x, adj_y):  # validate move & update state
        new_state = get_new_state(current.grid_state, current.blank_x, current.blank_y, adj_x, adj_y)

        state_tuple = tuple(map(tuple, new_state))  # convert new state to tuple, add to set

        if state_tuple not in SEEN:
          SEEN.add(state_tuple)   # add visited state to set, push state to stack
          Q.append(EightPuzzle(new_state, adj_x, adj_y, current.depth_level+1))

  print("Goal state unreachable from 8-Puzzle root state.")  # all possible moves exhausted

In [10]:
# DFS
def dfs_solver(grid_state, blank_x, blank_y):
  """
  Searches complete branch of nodes (8-puzzle states) before visiting sibling nodes.
  @param grid_state: Current state of 8-puzzle grid.
  @param blank_x: Row position of blank tile.
  @param blank_y: Column position of blank tile.
  """
  S = []        # LIFO stack for exploring branch depths (iteratively deepening)
  SEEN = set()  # re-init per instance, stores seen states to prevent cycling

  S.append(EightPuzzle(grid_state, blank_x, blank_y, 0))  # push root to stack
  SEEN.add(tuple(map(tuple, grid_state)))  # convert to tuple, add to set

  while S:              # while nodes (states) remain on stack
    current = S.pop()   # pop the most recently added node (state)

    print(f"\nDepth = {current.depth_level}:")  # print current depth
    to_grid(current.grid_state)                # print state as NxN grid

    if state_equals_goal(current.grid_state):
      print(f"\n~ Goal state reached at depth {current.depth_level}! ~")  # solution state reached
      return  # exit

    for i in range(4):                        # move blank tile
      adj_x = current.blank_x + row_moves[i]  # new x = current + L/R/U/D
      adj_y = current.blank_y + col_moves[i]  # new y = current + L/R/U/D

      if is_legal_move(adj_x, adj_y):  # validate move & update state
        new_state = get_new_state(current.grid_state, current.blank_x, current.blank_y, move_x, move_y)

        state_tuple = tuple(map(tuple, new_state))  # convert to tuples

        if state_tuple not in SEEN:
          SEEN.add(state_tuple)   # add visited state to set, push state to stack
          S.append(EightPuzzle(new_state, adj_x, adj_y, current.depth_level+1))  # add new state to stack with new x,y & incremented depth level

  print("Goal state unreachable from 8-Puzzle root state.")  # all possible moves exhausted

In [11]:
# Helper method to display menu & return validated user input
def get_menu_choice():
  while True:  # loop menu prompt until valid input
    print("\n-------------------------------------")
    print("| 8-PUZZLE BFS/DFS SOLVER MAIN MENU |")
    print("-------------------------------------")
    print("1. New random 8-Puzzle root state")
    print("2. Attempt to solve via BFS")
    print("3. Attempt to solve via Iteratively Deepening DFS")
    print("4. Exit program")

    choice = input("Enter a menu option: ").strip()  # save input without spaces

    if choice == '1' or choice == '2' or choice == '3' or choice == '4':
      return choice
    else:
      print("\nInvalid input. Enter a valid menu number: 1, 2, 3, or 4.")


In [12]:
# Driver UI
def main():
  puzzle = None      # init to null (prevent BFS or DFS without grid)
  x, y = None, None  # init to null

  while True:
    choice = get_menu_choice()  # display menu prompt & return valid input

    # MENU 1: GENERATE NEW 8-PUZZLE GRID
    if choice == '1':
      puzzle, x, y = get_root_state()  # get randomized grid & xy coords of value 0
      print("\nNew 8-Puzzle board created:")
      to_grid(puzzle)  # print initial puzzle state

    # MENU 2: SOLVE VIA BFS
    if choice == '2':
      if not puzzle:
        print("\nRequires 8-Puzzle board via menu option 1.")
      else:
        print()  # formatting
        bfs_solver(puzzle, x, y)  # call bfs, pass state & blank tile coords

    # Menu 3: Solve via DFS
    if choice == '3':
      if not puzzle:
        print("\nRequires 8-Puzzle board via menu option 1.")
      else:
        print()  # formatting
        dfs_solver(puzzle, x, y)  # call dfs, pass state & blank tile coords

    # Menu 4: Exit
    if choice == '4':
      print("\nExiting program... Goodbye!")
      return

In [ ]:
# Tester
if __name__ == '__main__':
  main()


-------------------------------------
| 8-PUZZLE BFS/DFS SOLVER MAIN MENU |
-------------------------------------
1. New random 8-Puzzle root state
2. Attempt to solve via BFS
3. Attempt to solve via Iteratively Deepening DFS
4. Exit program
